## Запуск

+ country - страна-производитель вина
+ description - подробное описание
+ designation - название виноградника, где выращивают виноград для вина
+ points - количество баллов, которое WineEnthusiast оценил вино по шкале от 1 до 100
+ price - стоимость бутылки вина
+ province - провинция или штат 
+ region_1 - винодельческий район в провинции или штате (например, Напа)
+ region_2 - конкретный регион. Иногда в пределах винодельческой зоны указываются более конкретные регионы (например, Резерфорд в долине Напа), но это значение иногда может быть пустым.
+ taster_name - имя дегустатора
+ taster_twitter_handle - твиттер дегустатора
+ title - название вина, который часто содержит годи др подробную информацию
+ variety -  сорт винограда, из которого изготовлено вино (например, Пино Нуар).
+ winery - винодельня, которая производила вино

In [53]:
import pandas as pd
import dtale


In [54]:
df = pd.read_csv('Data\wine_cleared.csv')  # чтение данных
print(f"Размер датасета: {df.shape}")
print("Первые 5 строк:")
df.head()

Размер датасета: (129971, 13)
Первые 5 строк:


,Unnamed: 0,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


## Создание признаков

### даты и времени

In [55]:
# для удобства сразу преобразуем признак в int 
df['price_round'] = df['price'].round().astype(int)

In [56]:
def extract_wine_year(title, designation):
    """
    Извлекает год производства вина из title, исключая годы из названия виноградника
    """
    import re
    regex = r'(1[8-9]\d{2}|20[0-2]\d)'
    
    # Находим все годы в title
    years_in_title = re.findall(regex, str(title))
    
    if not years_in_title:
        return None
    
    # Если designation не пустой, исключаем годы из него
    if pd.notna(designation) and designation != 'unknown':
        years_in_designation = re.findall(regex, str(designation))
        # Оставляем только те годы, которых нет в designation
        valid_years = [year for year in years_in_title if year not in years_in_designation]
        if valid_years:
            return valid_years[-1]  # берем последний валидный год
    
    # Если designation пустой или все годы валидные, берем последний год
    return years_in_title[-1]

df['year'] = df.apply(lambda row: extract_wine_year(row['title'], row['designation']), axis=1)

In [57]:
df['is_usa'] = df['country'].apply(lambda x: 1 if x == 'US' else 0)
df['is_italy'] = df['country'].apply(lambda x: 1 if x == 'Italy' else 0)
df['is_france'] = df['country'].apply(lambda x: 1 if x == 'France' else 0)
df['old_wine'] = df['year'].apply(lambda x: 1 if pd.notna(x) and int(x) < 2010 else 0)



In [58]:
# Анализ проблемных строк для проверки правильности выбора года
import re

def analyze_year_extraction(df):
    """
    Анализирует качество извлечения годов и показывает проблемные случаи
    """
    regex = r'(1[8-9]\d{2}|20[0-2]\d)'
    
    # Находим строки с множественными годами в title
    multiple_years = []
    designation_conflicts = []
    
    for idx, row in df.iterrows():
        title = str(row['title'])
        designation = str(row['designation'])
        extracted_year = row['year']
        
        # Находим все годы в title
        years_in_title = re.findall(regex, title)
        years_in_designation = re.findall(regex, designation) if designation != 'nan' and designation != 'unknown' else []
        
        # Случаи с множественными годами
        if len(years_in_title) > 1:
            multiple_years.append({
                'index': idx,
                'title': title,
                'designation': designation,
                'years_in_title': years_in_title,
                'years_in_designation': years_in_designation,
                'extracted_year': extracted_year
            })
        
        # Случаи с годами в designation
        if years_in_designation:
            designation_conflicts.append({
                'index': idx,
                'title': title,
                'designation': designation,
                'years_in_title': years_in_title,
                'years_in_designation': years_in_designation,
                'extracted_year': extracted_year
            })
    
    print("=== СТРОКИ С МНОЖЕСТВЕННЫМИ ГОДАМИ В TITLE ===")
    print(f"Найдено {len(multiple_years)} строк с несколькими годами")
    for i, case in enumerate(multiple_years[:10]):  # показываем первые 10
        print(f"\n{i+1}. Index: {case['index']}")
        print(f"   Title: {case['title']}")
        print(f"   Designation: {case['designation']}")
        print(f"   Годы в title: {case['years_in_title']}")
        print(f"   Годы в designation: {case['years_in_designation']}")
        print(f"   Извлеченный год: {case['extracted_year']}")
    
    print(f"\n=== СТРОКИ С ГОДАМИ В DESIGNATION ===")
    print(f"Найдено {len(designation_conflicts)} строк с годами в designation")
    for i, case in enumerate(designation_conflicts[:10]):  # показываем первые 10
        print(f"\n{i+1}. Index: {case['index']}")
        print(f"   Title: {case['title']}")
        print(f"   Designation: {case['designation']}")
        print(f"   Годы в title: {case['years_in_title']}")
        print(f"   Годы в designation: {case['years_in_designation']}")
        print(f"   Извлеченный год: {case['extracted_year']}")
    
    return multiple_years, designation_conflicts

# Запускаем анализ
multiple_years_cases, designation_conflicts_cases = analyze_year_extraction(df)


=== СТРОКИ С МНОЖЕСТВЕННЫМИ ГОДАМИ В TITLE ===
Найдено 304 строк с несколькими годами

1. Index: 1012
   Title: Antichi Vinai 1877 2013 Pietralava Red (Etna)
   Designation: Pietralava
   Годы в title: ['1877', '2013']
   Годы в designation: []
   Извлеченный год: 2013

2. Index: 1512
   Title: Pascal Aufranc 2011 Vignes de 1939  (Chénas)
   Designation: Vignes de 1939
   Годы в title: ['2011', '1939']
   Годы в designation: ['1939']
   Извлеченный год: 2011

3. Index: 2634
   Title: Caccia al Piano 1868 2009 Ruit Hora  (Bolgheri)
   Designation: Ruit Hora
   Годы в title: ['1868', '2009']
   Годы в designation: []
   Извлеченный год: 2009

4. Index: 2660
   Title: Donna Olimpia 1898 2009 Tageto Red (Toscana)
   Designation: Tageto
   Годы в title: ['1898', '2009']
   Годы в designation: []
   Извлеченный год: 2009

5. Index: 2858
   Title: San Pedro 2014 1865 Single Vineyard Cabernet Sauvignon (Maipo Valley)
   Designation: 1865 Single Vineyard
   Годы в title: ['2014', '1865']
   Год

In [59]:
# Дополнительный анализ: проверка качества извлечения года
def check_extraction_quality(df, sample_size=20):
    """
    Показывает случайную выборку для ручной проверки качества извлечения года
    """
    regex = r'(1[8-9]\d{2}|20[0-2]\d)'
    
    # Берем случайную выборку строк с годами
    sample_df = df[df['year'].notna()].sample(n=min(sample_size, len(df)), random_state=42)
    
    print("=== СЛУЧАЙНАЯ ВЫБОРКА ДЛЯ ПРОВЕРКИ КАЧЕСТВА ===")
    for idx, row in sample_df.iterrows():
        title = str(row['title'])
        designation = str(row['designation'])
        extracted_year = row['year']
        
        years_in_title = re.findall(regex, title)
        years_in_designation = re.findall(regex, designation) if designation != 'nan' and designation != 'unknown' else []
        
        print(f"\nIndex: {idx}")
        print(f"Title: {title}")
        print(f"Designation: {designation}")
        print(f"Все годы в title: {years_in_title}")
        print(f"Годы в designation: {years_in_designation}")
        print(f"Извлеченный год: {extracted_year}")
        print("-" * 80)

# Статистика по извлечению годов
def year_extraction_stats(df):
    """
    Показывает статистику извлечения годов
    """
    total_rows = len(df)
    rows_with_year = df['year'].notna().sum()
    rows_without_year = total_rows - rows_with_year
    
    print("=== СТАТИСТИКА ИЗВЛЕЧЕНИЯ ГОДОВ ===")
    print(f"Всего строк: {total_rows}")
    print(f"Строк с извлеченным годом: {rows_with_year} ({rows_with_year/total_rows*100:.1f}%)")
    print(f"Строк без года: {rows_without_year} ({rows_without_year/total_rows*100:.1f}%)")
    
    if rows_with_year > 0:
        year_counts = df['year'].value_counts().head(10)
        print(f"\nТоп-10 самых частых годов:")
        for year, count in year_counts.items():
            print(f"  {year}: {count} раз")
        
        # Проверяем диапазон годов
        years_numeric = pd.to_numeric(df['year'], errors='coerce')
        min_year = years_numeric.min()
        max_year = years_numeric.max()
        print(f"\nДиапазон годов: {int(min_year)} - {int(max_year)}")

# Запускаем дополнительный анализ
print("\\n" + "="*50)
year_extraction_stats(df)
print("\\n" + "="*50)
check_extraction_quality(df, sample_size=15)


\n==================================================
=== СТАТИСТИКА ИЗВЛЕЧЕНИЯ ГОДОВ ===
Всего строк: 129971
Строк с извлеченным годом: 125360 (96.5%)
Строк без года: 4611 (3.5%)

Топ-10 самых частых годов:
  2013: 15875 раз
  2012: 15747 раз
  2014: 15582 раз
  2011: 12558 раз
  2010: 12167 раз
  2015: 10052 раз
  2009: 9878 раз
  2008: 7439 раз
  2007: 7053 раз
  2006: 5772 раз

Диапазон годов: 1821 - 2017
\n==================================================
=== СЛУЧАЙНАЯ ВЫБОРКА ДЛЯ ПРОВЕРКИ КАЧЕСТВА ===

Index: 90682
Title: Quivira 2014 Black Boar Zinfandel (Dry Creek Valley)
Designation: Black Boar
Все годы в title: ['2014']
Годы в designation: []
Извлеченный год: 2014
--------------------------------------------------------------------------------

Index: 31335
Title: Robert Hall 2014 Merlot (Paso Robles)
Designation: unknown
Все годы в title: ['2014']
Годы в designation: []
Извлеченный год: 2014
--------------------------------------------------------------------------------

In

In [60]:
dtale.show(df)
